# Segmenting and Clustering Neighborhoods in Toronto

## Description

This project is to explore, segment, and cluster the neighborhoods in the city of Toronto.

**Content:**
- **Section 1:** Get Toronto Neighborhood Data from Wikipedia
- **Section 2:** Load Geographical Coordinates
- **Section 3:** Exploration & Clustering of Neighborhoods

## Section 1: Load Neighborhood Data (Web Scrapping)

Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

**Packages**:<br>
- **BeautifulSoup**: for web scrapping
- **requesets**: library to handle requests 

In [1]:
from bs4 import BeautifulSoup
import requests 
import lxml
import pandas as pd
import numpy as np

URL reques

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
web=BeautifulSoup(source,"lxml")

Find and save target content

In [3]:
body=web.find("tbody")
mylist=[]
for tr in body.find_all("tr"):
    for td in tr.find_all("td"):
        a=str(td.text.split("\n")[0])
        mylist.append(a)

Clean and save target content into a dataframe

To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [4]:
mylist = np.array(mylist)
mylist = mylist.reshape(int(len(mylist)/3),3)
delete =[]
for i in range(len(mylist)):
    if mylist[i][1] == "Not assigned":
        delete.append(i)
    if mylist[i][2] == "Not assigned":
        mylist[i][2]=mylist[i][1]
delete = list(set(delete))
mydf = pd.DataFrame(mylist)
mydf=mydf.drop(index=delete)
mydf.columns = ['PostalCode','Borough','Neighborhood']
mydf=mydf.sort_values(by = ['PostalCode'])
delete =[]
for i in range(len(mydf)-1,0,-1):
    if mydf.iloc[i][0]==mydf.iloc[i-1][0]:
        mydf.iloc[i-1][2]=mydf.iloc[i-1][2]+","+mydf.iloc[i][2]
        delete.append(i)
        
mydf.index=range(len(mydf))
mydf=mydf.drop(index=delete)
mydf

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
5,M1E,Scarborough,"Guildwood,Morningside,West Hill"
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae
10,M1J,Scarborough,Scarborough Village
11,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
14,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
17,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"
20,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [40]:
mydf.shape

(103, 3)

## Section 2: Load Geographical Coordinates

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

In [5]:
geo_table =pd.read_csv("https://cocl.us/Geospatial_data",delimiter=",",header = 0,index_col=0)

S1=[]
S2=[]
for i in mydf['PostalCode']:
    S1.append(geo_table.loc[i][0])
    S2.append(geo_table.loc[i][1])
mydf["Latitude"]=S1
mydf["Longitude"]=S2
mydf.index=range(len(mydf))
mydf

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


## Section 3: Exploration & Clustering of Neighborhoods

##### Explore and cluster the neighborhoods in Toronto.

**Packages**:<br>
- **folium**: map rendering library
- **Nominatim**: convert an address into latitude and longitude values
- **matplotlib**: Matplotlib and associated plotting modules
- **sklearn**: to implement k-means from clustering stage

In [7]:
import folium # map rendering library
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import sklearn

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>toronto_explorer</em>, as shown below.

In [8]:
address = "Toronto"
geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("the geographical coordinate of Toronto are {},{}.".format(latitude, longitude))

the geographical coordinate of Toronto are 43.653963,-79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location = [latitude, longitude],zoom_start = 11)

# add markers to map
for label,lat, lng  in zip(mydf["PostalCode"],mydf["Latitude"],mydf["Longitude"]):
    label =folium.Popup(label,parse_html=True)
    folium.CircleMarker(
            [lat,lng],
            radius=5,
            popup=label,
            color="blue",
            fill=True,
            fill_color="#3186cc",
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)

# Display map
display(map_toronto)

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

However, for illustration purposes, let's simplify the above map and segment and cluster only boroughs that contain the word Toronto . So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [10]:
my_toronto = mydf.loc[mydf["Borough"].isin(['Central Toronto','Downtown Toronto','West Toronto','East Toronto'])]

Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = '4SCK0KHQNA4I1ODGMOYRWFD3FADXJZEC2KL21205DS0XMGVE' # your Foursquare ID
CLIENT_SECRET = '054TEA5XWSDHTBN2DMGMCURHICV1SN4KUEE3ZROAFVVL1VP5' # your Foursquare Secret
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4SCK0KHQNA4I1ODGMOYRWFD3FADXJZEC2KL21205DS0XMGVE
CLIENT_SECRET:054TEA5XWSDHTBN2DMGMCURHICV1SN4KUEE3ZROAFVVL1VP5


#### Let's explore the first neighborhood in our dataframe.

In [12]:
example_info=my_toronto.iloc[0][["PostalCode","Borough","Neighborhood"]]
example_longitude=my_toronto.iloc[0]["Longitude"]
example_latitude=my_toronto.iloc[0]["Latitude"]
example_name=my_toronto.iloc[0]["Neighborhood"]
print("Latitude and Longitude values of {} are {},{}".format(example_name,example_latitude,example_longitude))

Latitude and Longitude values of The Beaches are 43.67635739999999,-79.2930312


Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.<br>
First, let's create the GET request URL. Name your URL **url**.

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    example_latitude, 
    example_longitude, 
    radius, 
    LIMIT)
print(url) # display URL

# Send the GET request and examine the resutls
results =requests.get(url).json()

https://api.foursquare.com/v2/venues/explore?&client_id=4SCK0KHQNA4I1ODGMOYRWFD3FADXJZEC2KL21205DS0XMGVE&client_secret=054TEA5XWSDHTBN2DMGMCURHICV1SN4KUEE3ZROAFVVL1VP5&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100


From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try: categories_list=row['categories']
    except:categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]["name"]

Now we are ready to clean the json and structure it into a *pandas* dataframe.


In [15]:
venues = results["response"]["groups"][0]["items"]
nearby_venues=json_normalize(venues)

# filter columns
filter_columns = ["venue.name","venue.categories","venue.location.lat","venue.location.lng"]
nearby_venues=nearby_venues.loc[:,filter_columns]

# filter the category for each row
nearby_venues["venue.categories"]=nearby_venues.apply(get_category_type,axis=1)

# clean columns
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
4,Upper Beaches,Neighborhood,43.680563,-79.292869


how many venues were returned by Foursquare?

In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


#### Explore Neighborhoods in Toronto

Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [17]:
def getNearbyVenues(names,latitudes, longitudes, radius = 500):
    venues_list = []
    for name, lat, lng in zip(names,latitudes, longitudes):        
        #create API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
        
        #make the GET request
        results =requests.get(url).json()["response"]["groups"][0]["items"]

        #return only relevant info for each nearby venue
        venues_list.append([(name,lat,lng,v['venue']['name'],
                             v['venue']['location']['lat'],
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood','Neighborhood Latitude',"Neighborhood Longitude","Venue","Venue Latitude",
                             "Venue Longitude",'Venue Category']
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [18]:
toronto_venues = getNearbyVenues(names=my_toronto["PostalCode"],
                                   latitudes=my_toronto["Latitude"],
                                   longitudes=my_toronto["Longitude"],
                                   radius = 500)
#check the size of resulting dataframe

In [19]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Let's check how many venues were returned for each neighborhood

In [20]:
#how many venues retruned for each neighborhood
toronto_venues.groupby("Neighborhood").count()["Venue"]

Neighborhood
M4E      5
M4K     41
M4L     17
M4M     40
M4N      4
M4P     10
M4R     20
M4S     34
M4T      3
M4V     15
M4W      5
M4X     46
M4Y     82
M5A     45
M5B    100
M5C    100
M5E     56
M5G     82
M5H    100
M5J    100
M5K    100
M5L    100
M5N      3
M5P      4
M5R     23
M5S     33
M5T    100
M5V     16
M5W     96
M5X    100
M6G     16
M6H     14
M6J     64
M6K     23
M6P     23
M6R     15
M6S     38
M7Y     16
Name: Venue, dtype: int64

Let's find out how many unique categories can be curated from all the returned venues

In [21]:
#show how many unique categories returned 
print("there are {} unique categories.".format(len(toronto_venues["Venue Category"].unique())))

there are 232 unique categories.


#### Analyze Each Neighborhood

In [23]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[["Venue Category"]], prefix = "",prefix_sep="")

#add neighborhood column back to dataframe
toronto_onehot["Neighborhood"]=toronto_venues["Neighborhood"]

#move this column to first column
fixed_columns = [toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [24]:
toronto_onehot.shape

(1689, 232)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,M4E,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.200000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,0.024390,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.024390,...,0.000000,0.00,0.000000,0.024390,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,M4L,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,0.025000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.050000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,M4N,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,0.050000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,M4S,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.000000,0.333333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.066667,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000


Let's confirm the new size

In [26]:
toronto_grouped.shape

(38, 232)

Let's print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
                  venue  freq
0     Health Food Store   0.2
1  Other Great Outdoors   0.2
2                   Pub   0.2
3                 Trail   0.2
4           Music Store   0.0


----M4K----
                    venue  freq
0        Greek Restaurant  0.22
1             Coffee Shop  0.10
2          Ice Cream Shop  0.07
3      Italian Restaurant  0.07
4  Furniture / Home Store  0.05


----M4L----
                venue  freq
0           Pet Store  0.06
1  Italian Restaurant  0.06
2   Fish & Chips Shop  0.06
3        Burger Joint  0.06
4       Burrito Place  0.06


----M4M----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3            Gastropub  0.05
4  American Restaurant  0.05


----M4N----
                venue  freq
0  Photography Studio  0.25
1                Park  0.25
2            Bus Line  0.25
3         Swim School  0.25
4              Museum  0.00


----M4P----
                  venue  freq
0     

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [28]:
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
   
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Other Great Outdoors,Trail,Pub,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Pizza Place,Bookstore,Brewery,Bubble Tea Shop
2,M4L,Pet Store,Italian Restaurant,Steakhouse,Sushi Restaurant,Brewery,Ice Cream Shop,Pub,Sandwich Place,Movie Theater,Fish & Chips Shop
3,M4M,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Bookstore,Fish Market,Cheese Shop,Latin American Restaurant
4,M4N,Photography Studio,Park,Bus Line,Swim School,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster Neighborhoods

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 0, 2, 2, 2, 1, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = my_toronto
toronto_merged['Neighborhoods'] = my_toronto['Neighborhood']
toronto_merged['Neighborhood'] = my_toronto['PostalCode']

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

C:\Users\sunka\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\sunka\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,M4E,43.676357,-79.293031,The Beaches,2,Health Food Store,Other Great Outdoors,Trail,Pub,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
41,M4K,East Toronto,M4K,43.679557,-79.352188,"The Danforth West,Riverdale",2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Pizza Place,Bookstore,Brewery,Bubble Tea Shop
42,M4L,East Toronto,M4L,43.668999,-79.315572,"The Beaches West,India Bazaar",2,Pet Store,Italian Restaurant,Steakhouse,Sushi Restaurant,Brewery,Ice Cream Shop,Pub,Sandwich Place,Movie Theater,Fish & Chips Shop
43,M4M,East Toronto,M4M,43.659526,-79.340923,Studio District,2,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Bookstore,Fish Market,Cheese Shop,Latin American Restaurant
44,M4N,Central Toronto,M4N,43.728020,-79.388790,Lawrence Park,0,Photography Studio,Park,Bus Line,Swim School,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Finally, let's visualize the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,Lawrence Park,0,Photography Studio,Park,Bus Line,Swim School,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
76,West Toronto,"Dovercourt Village,Dufferin",0,Pharmacy,Bakery,Supermarket,Gym / Fitness Center,Music Venue,Middle Eastern Restaurant,Café,Brewery,Bar,Bank


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,"Moore Park,Summerhill East",1,Playground,Trail,Summer Camp,Wings Joint,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,The Beaches,2,Health Food Store,Other Great Outdoors,Trail,Pub,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
41,East Toronto,"The Danforth West,Riverdale",2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Pizza Place,Bookstore,Brewery,Bubble Tea Shop
42,East Toronto,"The Beaches West,India Bazaar",2,Pet Store,Italian Restaurant,Steakhouse,Sushi Restaurant,Brewery,Ice Cream Shop,Pub,Sandwich Place,Movie Theater,Fish & Chips Shop
43,East Toronto,Studio District,2,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Bookstore,Fish Market,Cheese Shop,Latin American Restaurant
45,Central Toronto,Davisville North,2,Gym / Fitness Center,Hotel,Clothing Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Asian Restaurant,Park,Gym,Grocery Store
46,Central Toronto,North Toronto West,2,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Bagel Shop,Gift Shop,Metro Station,Mexican Restaurant,Diner,Dessert Shop
47,Central Toronto,Davisville,2,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Indian Restaurant,Japanese Restaurant,Farmers Market
49,Central Toronto,"South Hill,Rathnelly,Summerhill West,Deer Park...",2,Coffee Shop,Pub,American Restaurant,Liquor Store,Sports Bar,Bagel Shop,Supermarket,Restaurant,Sushi Restaurant,Fried Chicken Joint
51,Downtown Toronto,"Cabbagetown,St. James Town",2,Restaurant,Coffee Shop,Bakery,Flower Shop,Pub,Pizza Place,Italian Restaurant,Café,Beer Store,Dive Bar
52,Downtown Toronto,Church and Wellesley,2,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Gastropub,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,Roselawn,3,Garden,Home Service,Music Venue,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,Rosedale,4,Park,Playground,Trail,Building,Wings Joint,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
64,Central Toronto,"Forest Hill North,Forest Hill West",4,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Dive Bar,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
